In [4]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

import asyncio

In [5]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMOD
void reset_gpio(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpio(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}

In [8]:
reset_gpio()
for i in range(8):
    print(read_gpio(i))

0
0
1
1
0
0
1
1


In [ ]:
reset_gpio()

frequency = 10 #Can be any frequency, but probably want it at least in kHz range
duty_cycle = .5 #should be a percent from 0 to 1.
time_on = duty_cycle * (1/frequency)
time_off = (1-duty_cycle) * (1/frequency)

y=1
last = time.time()
while True:
    write_gpio(y,1)
    time.sleep(time_on)
    write_gpio(y,0)
    time.sleep(time_off)
    
    if time.time() - last > 1:
        last = time.time()
        print(f"time = {last}")
    


In [ ]:
#function for the keypad
def keypad():
    global sound, 
    reset_gpio()
    keypad_input = []
    
    for i in range(8): #see which two of the 8 pins are on!
        y = read.gpio(i)
        #if pin on, write to input array
        if y == 1: 
            keypad_input.append(i)

    if keypad_input[0] == 0:
        if keypad_input[1] == 4: #if row1, col1
            sound.append('R1C1')
        elif keypad_input[1] == 5: #if row1, col2
            sound.append('R1C2')
        elif keypad_input[1] == 6: #if row1, col3
            sound.append('R1C3')
        elif keypad_input[1] == 7: #if row1, col4
            sound.append('R1C4')
    elif keypad_input[0] == 1:
        if keypad_input[1] == 4: #if row2, col1
            sound.append('R2C1')
        elif keypad_input[1] == 5: #if row2, col2
            sound.append('R2C2')
        elif keypad_input[1] == 6: #if row2, col3
            sound.append('R2C3')
        elif keypad_input[1] == 7: #if row2 col4
            sound.append('R2C4')
    elif keypad_input[0] == 2:
        if keypad_input[1] == 4: #if row3, col1
            sound.append('R3C1')
        elif keypad_input[1] == 5: #if row3, col2
            sound.append('R3C2')
        elif keypad_input[1] == 6: #if row3, col3
            sound.append('R3C3')
        elif keypad_input[1] == 7: #if row3 col4
            sound.append('R3C4')
    elif keypad_input[0] == 1:
        if keypad_input[1] == 4: #if row4, col1
            sound.append('R4C1')
        elif keypad_input[1] == 5: #if row4, col2
            sound.append('R4C2')
        elif keypad_input[1] == 6: #if row4, col3
            sound.append('R4C3')
        elif keypad_input[1] == 7: #if row4 col4
            sound.append('R4C4')

In [13]:
Y = 0
X = 0
a = 0
b = 4
c = 1
d = 5

print('intial:', read_gpio(a), read_gpio(b))
reset_gpio()
print('reset to: ', read_gpio(a), read_gpio(b))

async def readinput(x0,y0):
    global X, Y
    while True:
        #X = read_gpio(x0)
        #Y = read_gpio(y0)
        print('readinput:', X, Y)
        read_gpio(x0)
        read_gpio(y0)
        reset_gpio()
        await asyncio.sleep(1)
    
async def readkeypad(_loop, x1, y1):
    global X, Y
    while True:
        await asyncio.sleep(.4)
        X = read_gpio(x1)
        Y = read_gpio(y1)
        if X == 1 & Y == 1:
            print('button A input', X, Y)
            reset_gpio()
            X = read_gpio(x1)
            Y = read_gpio(y1)
            print('Reset to:', X, Y)
        else:
            print('No input', X, Y)
            reset_gpio()
    

sound = []
LEDs = [base.leds[0] , base.leds[1] , base.leds[2] , base.leds[3]]

loop = asyncio.new_event_loop()
loop.create_task(readinput(a,b))
loop.create_task(readkeypad(loop, a, b))
loop.run_forever()

#while True:
    #print(read_gpio(1))
    #time.sleep(1)


intial: 0 0
reset to:  0 0
readinput: 0 0
No input 0 0
No input 0 0
readinput: 0 0
No input 0 0
No input 0 0
readinput: 0 0
No input 0 0
No input 0 0
No input 0 0
readinput: 0 0
No input 0 0
No input 0 0
readinput: 0 0


KeyboardInterrupt: 

In [9]:
Y = 0
X = 0
a = 1
b = 5

read_gpio(a)
read_gpio(b)

print('intial:', read_gpio(a), read_gpio(b))
reset_gpio()
print('reset to: ', read_gpio(a), read_gpio(b))
write_gpio(a, 0)
write_gpio(b,0)
print('manually reset to: ', read_gpio(a), read_gpio(b))
print('manually reset to: ', read_gpio(2), read_gpio(6))



intial: 1 1
reset to:  0 0
manually reset to:  0 0
manually reset to:  1 1


In [10]:
for i in range(8):
    print('initial', i, read_gpio(i))
    read_gpio(i)
    write_gpio(i,0)
    print('end: ', i, read_gpio(i))

initial 0 0
end:  0 0
initial 1 1
end:  1 0
initial 2 1
end:  2 1
initial 3 1
end:  3 1
initial 4 0
end:  4 0
initial 5 1
end:  5 0
initial 6 1
end:  6 1
initial 7 1
end:  7 1
